In [1]:
from ipywidgets import widgets, interact, fixed
from IPython.display import display
%matplotlib inline
import seaborn as sbn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from IPython.core.pylabtools import figsize
import scipy
import scipy.interpolate
from contextlib import redirect_stdout
from CoolProp import CoolProp
figsize(12, 10)
sbn.set_context("talk", font_scale=1)

In [2]:
pip install coolprop

Note: you may need to restart the kernel to use updated packages.


# Defining the function for the sensetivity analysis

In [98]:
@interact
def BV_Diff_O2(L=0.01, tfinal=21600, sat=0.15, i0=2.55, T=298., OV=-0.075):
    alpha = 2
    F = 96485
    R = 8.314
    #current = np.array([0])
    O2_limit = 10**-20
    CO20 = 0.26  # mol/m3
    COH0 = (10**-4)  # mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20  # Boundary condition 1, in the bulk

    # Diffusion
    N = 20  # number of points to discretize
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing

    D = 2.9 * 10**-9  # Diffusivity

    Ntsteps = tfinal
    dt = tfinal / (Ntsteps)
    t = np.arange(0, tfinal, dt)
    alpha = D * dt / h**2

    C = np.zeros(X.shape)
    C[0] = BC1

    #current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat:
            N[-1] = N[-2]
            #current = np.append(current, 0)
            concOH = np.append(concOH, concOH[-1])
            concO2 = np.append(concO2, N[-1])
        else:
            iloc = i0 * ((concOH[j] * np.exp((alpha * F * OV) / (R * T))) - (N[-2] * np.exp((-(alpha) * F * OV) / (R * T))))
            RO2 = -1 * abs(iloc) / (F * 4)
            N[-1] = (RO2 * h) + N[-2]
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4 * abs(iloc) / (F * 4)
            new_concOH = (ROH * h) + concOH[j]
            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc * 0.0104
            #current = np.append(current, newCurrent)

        C[:] = N

    return C[-1]



interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

Let's see the interaction for pH change as well:

In [25]:
@interact
def BV_Diff_pH( L = 0.01, #...
                  tfinal = 21600, # ...
                  sat = 0.15,
                  i0 = 2.55,
                  T = 298., # ...
                  OV = -0.075, # ...
                 ):
    alpha = 2#...
    F = 96485#...
    R = 8.314#...
    #current = np.array([0])
    O2_limit = 10**-20
#     OV = -0.075#
    CO20 = 0.26 #mol/m3
    COH0 = (10**-4)#mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20 #Boundary condition 1, in the bulk

    #   Diffusion
    N = 20  # number of points to discretize
    #L = 0.01
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing


    D = 2.9*10**-9  # Diffusivity

    #tfinal = 54000 #15 hours
    Ntsteps = tfinal
    dt = tfinal / (Ntsteps)
    t = np.arange(0, tfinal, dt)
    alpha = D * dt / h**2


    # initial condition at t = 0
    C = np.zeros(X.shape)
    C[0] = BC1

    #current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat: #to give it some time until the solution gets to a degree of saturation
            N[-1] = N[-2]  # derivative boundary condition flux = 0
            #current = np.append(current, 0)
            concOH = np.append(concOH, concOH[0])
            concO2 = np.append(concO2, N[-1])

        else: #after the solution is saturated with O2
            #print('Saturated')
            iloc = i0*((concOH[j]*np.exp((alpha*F*OV)/(R*T))) - (N[-2]*np.exp((-(alpha)*F*OV)/(R*T)))) #Current density

            RO2 = -1*abs(iloc)/(F*4) #Rate of O2 consumption or flux at the surface 

            N[-1] = (RO2*h) + N[-2]  # derivative boundary condition flux = RO2, BC2
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4*abs(iloc)/(F*4) #Rate of OH production

            new_concOH = (ROH*h) + concOH[j]

            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc*0.0104

            #current = np.append(current, newCurrent)

        C[:] = N #Crucial for the loop
     
    pH = 14 + np.log(concOH[tfinal] )
    pH0=14+np.log(COH0)
    pHChange=pH-pH0
    return  pHChange

interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

In [57]:
@interact
def BV_Diff_Current( L = 0.01, #...
                  tfinal = 21600, # ...
                  sat = 0.15,
                  i0 = 2.55,
                  T = 298., # ...
                  OV = -0.075, # ...
                 ):
    alpha = 2#...
    F = 96485#...
    R = 8.314#...
    current = np.array([0])
    O2_limit = 10**-20
#     OV = -0.075#
    CO20 = 0.26 #mol/m3
    COH0 = (10**-4)#mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20 #Boundary condition 1, in the bulk

    #   Diffusion
    N = 20  # number of points to discretize
    #L = 0.01
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing


    D = 2.9*10**-9  # Diffusivity

    #tfinal = 54000 #15 hours
    Ntsteps = tfinal
    dt = tfinal / (Ntsteps)
    t = np.arange(0, tfinal, dt)
    alpha = D * dt / h**2


    # initial condition at t = 0
    C = np.zeros(X.shape)
    C[0] = BC1

    current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < 0.15: #to give it some time until the solution gets to a degree of saturation
            N[-1] = N[-2]  # derivative boundary condition flux = 0
            current = np.append(current, 0)
            concOH = np.append(concOH, concOH[0])
            concO2 = np.append(concO2, N[-1])

        else: #after the solution is saturated with O2
            #print('Saturated')
            iloc = i0*((concOH[j]*np.exp((alpha*F*OV)/(R*T))) - (N[-2]*np.exp((-(alpha)*F*OV)/(R*T)))) #Current density

            RO2 = -1*abs(iloc)/(F*4) #Rate of O2 consumption or flux at the surface 

            N[-1] = (RO2*h) + N[-2]  # derivative boundary condition flux = RO2, BC2
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4*abs(iloc)/(F*4) #Rate of OH production

            new_concOH = (ROH*h) + concOH[j]

            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc*0.0104

            current = np.append(current, newCurrent)

        C[:] = N #Crucial for the loop
     
    
    return  current[-1]

interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

In [27]:
tfinal = 21600
Ntsteps = 21600  # Choose an appropriate number of steps
dt = tfinal / (Ntsteps)  # Calculate the correct time step
t = np.arange(0, tfinal, dt)
print(t)


[0.0000e+00 1.0000e+00 2.0000e+00 ... 2.1597e+04 2.1598e+04 2.1599e+04]


In [162]:

def BV_Diff_O2_new(L=0.01, tfinal=21600, sat=0.15, i0=2.55, T=298., OV=-0.075):
    alpha = 2
    F = 96485
    R = 8.314
    #current = np.array([0])
    O2_limit = 10**-20
    CO20 = 0.26  # mol/m3
    COH0 = (10**-4)  # mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20  # Boundary condition 1, in the bulk

    # Diffusion
    N = 20  # number of points to discretize
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing

    D = 2.9 * 10**-9  # Diffusivity
    
#     tfinal_new = tfinal.astype(int)
      
#     Ntsteps = tfinal_new
#     dt = tfinal_new / (Ntsteps)
    dt = 1
    print(tfinal_new)
    tfinal = np.array([18900,18900,19000])
    t = np.arange(0, tfinal, dt)
    alpha = D * dt / h**2

    C = np.zeros(X.shape)
    C[0] = BC1

    #current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat:
            N[-1] = N[-2]
            #current = np.append(current, 0)
            concOH = np.append(concOH, concOH[-1])
            concO2 = np.append(concO2, N[-1])
        else:
            iloc = i0 * ((concOH[j] * np.exp((alpha * F * OV) / (R * T))) - (N[-2] * np.exp((-(alpha) * F * OV) / (R * T))))
            RO2 = -1 * abs(iloc) / (F * 4)
            N[-1] = (RO2 * h) + N[-2]
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4 * abs(iloc) / (F * 4)
            new_concOH = (ROH * h) + concOH[j]
            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc * 0.0104
            #current = np.append(current, newCurrent)

        C[:] = N

    return C[-1]


In [163]:
a=BV_Diff_O2_new(L=0.01, tfinal=21600, sat=0.15, i0=2.55, T=298., OV=-0.075)
a

[18900 18900 18900 10800 10800 10800 10800]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Sensitivity Analysis

In [140]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp

### Define a problem file

In [141]:
morris_problem = {
    # There are six variables
    'num_vars': 6,
    # These are their names
    'names': ['L', 'tfinal', 'sat', 'i0', 'T', 'OV'],
    # These are their plausible ranges over which we'll move the variables
    'bounds': [[0.005, 0.05], # 'Double layer', 'm'
               [10800, 21600], # 'time duration','sec'
               [0.1, 0.2], # saturation degree', 'mol/m3'z
               [1.55, 3.55], #'exchange current density', 'A'
               [283, 313], # 'Temperature', 'K'
               [-0.1, -0.02], # 'over potential', 'v''
              ],
    # I don't want to group any of these variables together
     'groups': None
    }

In [142]:
morris_problem

{'num_vars': 6,
 'names': ['L', 'tfinal', 'sat', 'i0', 'T', 'OV'],
 'bounds': [[0.005, 0.05],
  [10800, 21600],
  [0.1, 0.2],
  [1.55, 3.55],
  [283, 313],
  [-0.1, -0.02]],
 'groups': None}

# Generate a Sample

In [143]:
num_levels = 3
trajectories = int(1e0)
sample = ms.sample(morris_problem, trajectories, num_levels=num_levels)
sample.shape

(7, 6)

In [125]:
tfinal = sample.T[1]
# tfinal = np.array([10.2, 10.4, 11.9])

# tfinal_new = np.array([]);
tfinal_new = tfinal.astype(int)
# for t in tfinal:
#     tfinal_new = np.append(tfinal_new,int(t))

# a = 5.4
# print(int(a))
print(tfinal_new)

[18900 18900 18900 10800 10800 10800 10800]


### Factor Prioritisation

We'll run a sensitivity analysis of the power module to see which is the most influential parameter.

The results parameters are called **mu**, **sigma** and **mu_star**.

* **Mu** is the mean effect caused by the input parameter being moved over its range.
* **Sigma** is the standard deviation of the mean effect.
* **Mu_star** is the mean absolute effect.

In [147]:
# Run the sample through the power model
output = BV_Diff_O2_new(*sample.T)
# print(output.shape)
# print(output)

[18900. 18900. 18900. 18900. 10800. 10800. 10800.]


TypeError: only size-1 arrays can be converted to Python scalars

In [54]:
# Store the results for plotting of the analysis
Si = ma.analyze(morris_problem, 
                sample, 
                output, 
                print_to_console=False, 
                num_levels=num_levels)
print("{:20s} {:>7s} {:>7s} {:>7s}".format("Name", "mu", "mu_star", "sigma"))
for name, s1, st, mean in zip(morris_problem['names'], Si['mu'], Si['mu_star'], Si['sigma']):
    print("{:20s} {:=7.2f} {:=7.2f} {:=7.2f}".format(name, s1, st, mean))

NameError: name 'output' is not defined

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
mp.horizontal_bar_plot(ax1, Si) #  param_dict={}
mp.covariance_plot(ax2, Si, {})

In [40]:
# Run nominal case
values_dict = {i.name: i.nominal for i in variables}
#values_dict.update(fixed_parameter_values)
print(values_dict)
result = BV_Diff(**values_dict)
print(result)

{'L': 0.005, 'tfinal': 10800, 'sat': 0.1, 'i0': 1.55, 'T': 283, 'OV': -0.1}
-0.004731988995257445


In [47]:
def run_many_simulations(data):
    results = []
    with open('simulations.log', 'w') as f:
        for i,row in enumerate(data):
            with redirect_stdout(f):
                values_dict = {p.name: v for p, v in zip(variables, row)}
                #values_dict.update(fixed_parameter_values)
                print(values_dict)
                try:
                    result = BV_Diff(**values_dict)
                except RuntimeError:
                    result = np.nan
                results.append(result)
                print(result)
                print('-'*80)
            print("Run {} of {} gives {}".format(i+1, len(data), result))
    return np.array(results)

# Morris Method

In [42]:
number_of_samples = 1000
sample = ms.sample(salib_problem, number_of_samples)
print(sample)
len(sample)

[[ 2.33333333e-02  2.88000000e+04  1.50000000e-01  3.21666667e+00
   3.13000000e+02 -5.66666667e-02]
 [ 2.33333333e-02  4.32000000e+04  1.50000000e-01  3.21666667e+00
   3.13000000e+02 -5.66666667e-02]
 [ 2.33333333e-02  4.32000000e+04  1.50000000e-01  3.21666667e+00
   3.03000000e+02 -5.66666667e-02]
 ...
 [ 3.66666667e-02  2.88000000e+04  1.66666667e-01  2.55000000e+00
   3.03000000e+02 -3.83333333e-02]
 [ 3.66666667e-02  2.88000000e+04  1.66666667e-01  2.55000000e+00
   3.13000000e+02 -3.83333333e-02]
 [ 3.66666667e-02  4.32000000e+04  1.66666667e-01  2.55000000e+00
   3.13000000e+02 -3.83333333e-02]]


7000

In [43]:
sample.shape

(7000, 6)

In [52]:
%%time
output = run_many_simulations(sample)
#output = run_many_simulations_parallel(sample)

NameError: name 'run_many_simulations' is not defined